# 0 - Librerías y variables

In [1]:
# Librerías
# ------------------------------------------------------------------------------
import os
import requests
import json

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [2]:
# Variables
# ------------------------------------------------------------------------------
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1 - Introducción a los RAG

RAG significa Retrieval-Augmented Generation, o Generación Aumentada por Recuperación. Es una arquitectura híbrida que combina LLMs con motores de recuperación de información, generalmente basados en embeddings y búsqueda vectorial.

La idea clave es no depender solo del conocimiento interno del LLM, sino complementarlo con información externa relevante que se recupera en tiempo real desde una base de datos documental.

**¿Por qué usar RAG?**

Los modelos generativos, aunque son muy potentes, tienen tres limitaciones clave:
- Alucinaciones: inventan respuestas cuando no saben algo.
- Obsolescencia: su conocimiento está limitado al momento en que fueron entrenados.
- Coste de fine-tuning: actualizar su conocimiento requiere reentrenar o usar técnicas más complejas.

RAG soluciona esto al buscar primero información en una base de datos externa (documentos, PDFs, páginas web, etc.), y luego pasar esa información como contexto al modelo LLM, que produce una respuesta más precisa, basada en datos.

**¿Cómo funciona RAG? (flujo típico)**

Un sistema RAG moderno, como el que puedes construir con LangChain, suele organizarse en las siguientes etapas:

- Load (Carga de documentos): Se ingestan las fuentes de información que van a alimentar al sistema. El objetivo es leer los datos en bruto y convertirlos en texto plano.
- Split (Segmentación de texto): Los textos se dividen en fragmentos más pequeños para facilitar su uso como contexto. Se usan técnicas de ventana deslizante con solapamiento para preservar la coherencia semántica.
- Storage (Vectorización y almacenamiento): Cada fragmento se convierte en un embedding y se almacena en una base vectorial para búsquedas por similitud semántica.
- Retrieval (Recuperación): La consulta del usuario se vectoriza y se buscan los fragmentos más similares en la base vectorial. Estos fragmentos forman el contexto que se pasará al modelo generativo.
- Output (Generación de respuesta):Se combinan el contexto recuperado y la consulta original, y se pasan al modelo de lenguaje, que genera una respuesta informada.


![](../img/RAG.png)

# 2 - Document Loading

El primer paso en cualquier sistema RAG es **ingestar los documentos que contienen el conocimiento base**. En LangChain, esta tarea se gestiona mediante los **loaders**, que permiten leer datos desde múltiples formatos y fuentes (PDFs, Word, Markdown, HTML, páginas web, etc.).

El objetivo es transformar estas fuentes heterogéneas en una representación uniforme: una lista de objetos `Document`, cada uno con su contenido de texto y metadatos asociados.

LangChain ofrece una interfaz común para todos los loaders, lo que facilita trabajar con diferentes tipos de datos sin tener que preocuparte por los detalles de bajo nivel.

> A partir de aquí, el texto ya puede ser segmentado y convertido en embeddings para la recuperación semántica.

En esta sección exploraremos cómo utilizar algunos de los loaders más comunes y cómo inspeccionar el resultado de la carga.

In [3]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2.1. - PDF

In [6]:
# Instanciar PDFLoader
loader = PyPDFLoader("../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf")

# Cargar PDF
pages = loader.load()

Cada página es un `Document` de LangChain, y cada `Document` contiene:
- contenido: `page_content`
- metadata: `metadata`

In [7]:
print(f"Páginas del PDF: {len(pages)}")

Páginas del PDF: 22


In [8]:
page = pages[0]

print(type(page))
page

<class 'langchain_core.documents.base.Document'>


Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': '../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \n

In [9]:
page.metadata

{'producer': 'Acrobat Distiller 8.1.0 (Windows)',
 'creator': 'PScript5.dll Version 5.2.2',
 'creationdate': '2008-07-11T11:25:23-07:00',
 'author': '',
 'moddate': '2008-07-11T11:25:23-07:00',
 'title': '',
 'source': '../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf',
 'total_pages': 22,
 'page': 0,
 'page_label': '1'}

In [10]:
page.page_content

'MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thing in computer science, but \nthe most exciting thing in all of human endeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine learning and all aspects of machine learning. Paul Baumstarck \nworks i

## 2.2. URLs

In [19]:
# Instanciar WebLoader
loader = WebBaseLoader("https://raw.githubusercontent.com/gmachinromero/exploring-langchain/refs/heads/master/README.md")

# Cargar URL
docs = loader.load()

In [20]:
len(docs)

1

In [21]:
doc = docs[0]

print(type(doc))
doc

<class 'langchain_core.documents.base.Document'>


Document(metadata={'source': 'https://raw.githubusercontent.com/gmachinromero/exploring-langchain/refs/heads/master/README.md'}, page_content='# Exploring LangChain! 🦜️🔗\n\nEste repositorio contiene diversas pruebas y experimentos realizados para aprender y explorar las capacidades del framework [LangChain](https://langchain.com/). Aquí encontrarás ejemplos de cómo utilizar las distintas herramientas que proporciona LangChain para la creación de aplicaciones basadas en inteligencia artificial, flujos de trabajo y manejo de grandes lenguajes de modelo (LLM).\n\n\n## Contenido\n\nEl contenido del repositorio permite ejecutar diferentes scripts de Python con el framework de Langchain con la idea de explorar:\n- Utilizar LLM tanto de forma online como en local\n- Conexiones a APIs\n- Desarrollo de agentes (Agents)\n- Desarrollo de herramientas (Tools)\n- Trazabilidad con LangSmith \n\n\n## Resultado\n\nEl resultado del proyecto es desplegar un frontal en local, que dado el nombre de un usu

In [22]:
doc.metadata

{'source': 'https://raw.githubusercontent.com/gmachinromero/exploring-langchain/refs/heads/master/README.md'}

# 3 - Document Splitting

Una vez que los documentos han sido cargados correctamente, el siguiente paso es **dividir el texto en fragmentos más pequeños**, adecuados para su posterior vectorización y uso como contexto en el sistema RAG.

Esto es necesario porque:

- Los modelos de lenguaje tienen un límite de tokens por entrada (context length).
- Dividir bien el texto ayuda a preservar la coherencia semántica.
- Mejora la calidad de la recuperación al indexar unidades más específicas de información.

LangChain proporciona varios **text splitters** que permiten segmentar el contenido de forma flexible. La elección del splitter y de sus parámetros (longitud, solapamiento, método de segmentación) **impacta directamente en la calidad del sistema RAG**.

| **Text Splitter**                  | **¿Para qué sirve?**                                                                                                                                                 |
|------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `RecursiveCharacterTextSplitter`   | El splitter más versátil. Divide el texto intentando respetar estructuras jerárquicas (párrafos, frases, palabras). Ideal para mantener coherencia semántica.        |
| `CharacterTextSplitter`            | Divide el texto en fragmentos basados en un número fijo de caracteres. Más simple que el `Recursive`, útil cuando no se necesita segmentación inteligente.           |
| `TokenTextSplitter`                | Divide el texto en función del número de tokens, en lugar de caracteres. Muy útil si trabajas con límites de contexto medidos en tokens (como modelos OpenAI).       |
| `MarkdownHeaderTextSplitter`       | Diseñado específicamente para documentos Markdown. Usa los encabezados (`#`, `##`, etc.) como guías de segmentación jerárquica.                                      |
| `NLTKTextSplitter`                 | Utiliza el tokenizer de NLTK para dividir el texto por frases u oraciones. Requiere instalar NLTK. Apto para NLP clásico o textos con puntuación bien estructurada.  |
| `SpacyTextSplitter`                | Usa spaCy para dividir por frases u oraciones gramaticales. Más preciso que NLTK pero también más pesado.                                                            |


En esta sección exploraremos cómo:

- Utilizar splitters como `RecursiveCharacterTextSplitter`.
- Controlar la longitud (`chunk_size`) y solapamiento de los fragmentos (`chunk_overlap`).
- Evaluar si los splits generados mantienen el significado y el contexto necesarios.

> La calidad del *splitting* es tan importante como la del *retrieval* — fragmentos mal segmentados pueden arruinar incluso los mejores embeddings.

In [114]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

## 3.1. - RecursiveCharacterTextSplitter

El RecursiveCharacterTextSplitter usa varios niveles de separación y además su lógica es recursiva, con un orden de preferencia de separadores:
- Saltos de línea dobles o simples
- Puntuación fuerte (. ; :)
- Comas y espacios

Si no puede dividir con lo anterior, corta por tamaño fijo (caracteres)

![](../img/splitter_chunk_size_overlap.png)

In [128]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=26,
    chunk_overlap=4
)

In [131]:
text1 = 'abcdefghijk lmnopqrstuvwxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnopq. rstuvwxyz'
r_splitter.split_text(text1)

['abcdefghijk',
 'lmnopqrstuvwxyzabcdefghij',
 'ghijklmnopqrstuvwxyzabcdef',
 'cdefghijklmnopq.',
 'rstuvwxyz']

In [132]:
text2 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
r_splitter.split_text(text2)

['a b c d e f g h i j k l m', 'l m n o p q r s t u v w x', 'w x y z']

In [87]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

len(some_text)

496

In [111]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)

r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example, closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.",
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [112]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "\. ", " ", ""]
)

r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [113]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

r_splitter.split_text(some_text)

["When writing documents, writers will use document structure to group content. This can convey to the reader, which idea's are related. For example,",
 'closely related ideas are in sentances. Similar ideas are in paragraphs. Paragraphs form a document.',
 'Paragraphs are often delimited with a carriage return or two carriage returns. Carriage returns are the "backslash n" you see embedded in this',
 'string. Sentences have a period at the end, but also, have a space.and words are separated by space.']

In [116]:
loader = PyPDFLoader("../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf")
pages = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

docs = text_splitter.split_documents(pages)

In [120]:
print(f"Páginas del PDF: {len(pages)}")
print(f"Chunks del PDF: {len(docs)}")

Páginas del PDF: 22
Chunks del PDF: 78


In [127]:
for i, doc in enumerate(docs[:2]):
    print(f"Chunk: {i}")
    print(doc.metadata)
    print(doc.page_content)

Chunk: 0
{'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': '../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}
MachineLearning-Lecture01  
Instructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine 
learning class. So what I wanna do today is just spend a little time going over the logistics 
of the class, and then we'll start to talk a bit about machine learning.  
By way of introduction, my name's Andrew Ng and I'll be instructor for this class. And so 
I personally work in machine learning, and I've worked on it for about 15 years now, and 
I actually think that machine learning is the most exciting field of all the computer 
sciences. So I'm actually always excited about teaching this class. Sometimes I actually 
think that machine learning is not on